<a href="https://colab.research.google.com/github/K-Godot/Orga_Innov/blob/main/W1_Creating_MeSH_GBD_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First, upload the following 'CSV' files to the 'content' directory.
1. IHME_GBD_2019_A1_HIERARCHIES_Y2020M10D15.csv
2. Crosswalk(S-16).csv
3. mesh_info.csv

# **Crosswalk (S-16)**

Crosswalk Cleansing

In [1]:
import csv

f = open('Crosswalk(S-16).csv', 'r')
reader = csv.reader(f)

whole_str = ''

for row in reader:
    whole_str+=row[0]
 
whole_str = whole_str.replace("??", "'") # Alzheimer's

In [2]:
whole_list = whole_str.split('?')[1:]

In [3]:
import pandas as pd

data = {
    'GBD disease cause': [],
    'MeSH code': [],
    'MeSH term': []
}

for item in whole_list:
  GBD_disease_cause = item.split('[')[0].strip()
  MeSH_code = item.split('[')[1].split(']')[0]
  MeSH_term = item.split('[')[1].split(']')[1].strip()

  data['GBD disease cause'].append(GBD_disease_cause)
  data['MeSH code'].append(MeSH_code)
  data['MeSH term'].append(MeSH_term)

df = pd.DataFrame(data)
df = df[~df['MeSH code'].str.contains("N/A")]

df.to_csv('Crosswalk(S-16)_cleansed.csv', index=False)

Data Exploration

In [4]:
## Filtering MeSH code with "C"
df_crosswalk = pd.read_csv('Crosswalk(S-16)_cleansed.csv')

df_C = df_crosswalk[df_crosswalk['MeSH code'].str.contains('C')]
df_C

,GBD disease cause,MeSH code,MeSH term
0,Acne vulgaris,C17.800.030.150,Acne Vulgaris
1,Acne vulgaris,C17.800.794.111,Acne Vulgaris
2,Acute glomerulonephritis,C12.777.419.570.363,Glomerulonephritis
3,Acute glomerulonephritis,C13.351.968.419.570.363,Glomerulonephritis
4,Acute hepatitis A,C02.440.420,Hepatitis A
...,...,...,...
601,Yellow fever,C02.081.980,Yellow Fever
602,Yellow fever,C02.782.350.250.980,Yellow Fever
603,Yellow fever,C02.782.417.881,Yellow Fever
605,Zika virus,C02.081.990,Zika Virus Infection


In [5]:
## Unique GBD disease causes
unique_cause = df_C['GBD disease cause'].unique()
print('Unique Diease Causes with MeSH code "C":', len(unique_cause))
print('Unique MeSH code "C":', len(df_C['MeSH code'].unique()))

Unique Diease Causes with MeSH code "C": 220
Unique MeSH code "C": 537


# **GBD Hierarchies Data**

Checking What Cause Outline "C" Means

In [ ]:
pd.set_option('display.max_columns', None)

df_GBD = pd.read_csv('GBD_hierarchies.csv')

GBD_col = ['Cause ID', 'Cause Name', 'Level', 'Cause Outline']
df_GBD_C = df_GBD[df_GBD['Cause Outline'].str.contains('C')][GBD_col] # 'C' seems irrelevant with MeSH code

Counting Unique Disease Causes

In [ ]:
## All causes are unique
df_GBD_sel = df_GBD[GBD_col]
print('Unique Disease Cause:', len(df_GBD['Cause Name'].unique()))
print('Simple Counting:', len(df_GBD['Cause Name']))

Unique Disease Cause: 360
Simple Counting: 360


Matching Crosswalk with GBD Data

In [ ]:
GBD_causes = list(df_GBD_sel['Cause Name'])
match = []

for cause in GBD_causes:
  if cause in list(unique_cause):
    match.append('True')
  else:
    match.append('False')

df_GBD_sel['Crosswalk Match'] = match
df_GBD_sel

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Cause ID,Cause Name,Level,Cause Outline,Crosswalk Match
0,294,All causes,0,Total,False
1,295,"Communicable, maternal, neonatal, and nutritio...",1,A,False
2,955,HIV/AIDS and sexually transmitted infections,2,A.1,True
3,298,HIV/AIDS,3,A.1.1,True
4,948,HIV/AIDS - Drug-susceptible Tuberculosis,4,A.1.1.1,False
...,...,...,...,...,...
355,726,Physical violence by sharp object,4,C.3.2.2,False
356,941,Sexual violence,4,C.3.2.3,False
357,727,Physical violence by other means,4,C.3.2.4,False
358,945,Conflict and terrorism,3,C.3.3,False


GBD Data that Matches Crosswalk

In [ ]:
df_GBD_match = df_GBD_sel[df_GBD_sel['Crosswalk Match'].str.contains('True')]
df_GBD_match

,Cause ID,Cause Name,Level,Cause Outline,Crosswalk Match
2,955,HIV/AIDS and sexually transmitted infections,2,A.1,True
3,298,HIV/AIDS,3,A.1.1,True
9,394,Syphilis,4,A.1.2.1,True
10,395,Chlamydial infection,4,A.1.2.2,True
11,396,Gonococcal infection,4,A.1.2.3,True
...,...,...,...,...,...
308,616,G6PD deficiency,4,B.12.4.5,True
309,839,G6PD trait,4,B.12.4.6,True
311,619,"Endocrine, metabolic, blood, and immune disorders",3,B.12.5,True
312,680,Oral disorders,3,B.12.6,True


In [ ]:
in_crosswalk_not_in_GBD = list(unique_cause)

for matched in list(df_GBD_match['Cause Name']):
  in_crosswalk_not_in_GBD.remove(matched) 

print('7 Disease Causes are Only in Crosswalk and not in GBD:')
print(in_crosswalk_not_in_GBD)

7 Disease Causes are Only in Crosswalk and not in GBD:
['Blindness and vision impairment', 'Brain and nervous system cancer', 'Epilepsy', 'H influenzae type B meningitis', 'Meningococcal meningitis', 'Myelodysplastic, myeloproliferative, and other hematopoieticneoplasms', 'Pneumococcal meningitis']


GBD Data that Does *NOT* Match Crosswalk

In [ ]:
df_GBD_no_match = df_GBD_sel[df_GBD_sel['Crosswalk Match'].str.contains('False')]
df_GBD_no_match

,Cause ID,Cause Name,Level,Cause Outline,Crosswalk Match
0,294,All causes,0,Total,False
1,295,"Communicable, maternal, neonatal, and nutritio...",1,A,False
4,948,HIV/AIDS - Drug-susceptible Tuberculosis,4,A.1.1.1,False
5,949,HIV/AIDS - Multidrug-resistant Tuberculosis wi...,4,A.1.1.2,False
6,950,HIV/AIDS - Extensively drug-resistant Tubercul...,4,A.1.1.3,False
...,...,...,...,...,...
355,726,Physical violence by sharp object,4,C.3.2.2,False
356,941,Sexual violence,4,C.3.2.3,False
357,727,Physical violence by other means,4,C.3.2.4,False
358,945,Conflict and terrorism,3,C.3.3,False


# **mesh_info.csv Data**

Filtering MeSH Code "C" from mesh_info.csv

In [7]:
mesh = pd.read_csv('mesh_info.csv')
mesh_C = mesh[mesh['mesh_number'].str.contains('C')]
mesh_C

,name,date_of_entry,unique_identifier,mesh_number
137,22q11 Deletion Syndrome,20100625,D058165,C16.320.180.019
138,22q11 Deletion Syndrome,20100625,D058165,C16.131.240.400.021
139,22q11 Deletion Syndrome,20100625,D058165,C19.642.482.500
140,22q11 Deletion Syndrome,20100625,D058165,C14.240.400.021
141,22q11 Deletion Syndrome,20100625,D058165,C16.131.077.019
...,...,...,...,...
56324,von Willebrand Diseases,19990101,D014842,C15.378.100.141.900
56325,von Willebrand Diseases,19990101,D014842,C15.378.140.900
56326,von Willebrand Diseases,19990101,D014842,C15.378.463.920
56327,von Willebrand Diseases,19990101,D014842,C16.320.099.920


In [10]:
print('Unique Diease Causes with MeSH code "C":', len(mesh_C['name'].unique()))
print('Unique MeSH code "C":', len(mesh_C['mesh_number'].unique()))

Unique Diease Causes with MeSH code "C": 4663
Unique MeSH code "C": 11400


# **Integrated Map Ver.1 (GBD hierarchies & Crosswalk(S-16))**

In [ ]:
df_C

,GBD disease cause,MeSH code,MeSH term
0,Acne vulgaris,C17.800.030.150,Acne Vulgaris
1,Acne vulgaris,C17.800.794.111,Acne Vulgaris
2,Acute glomerulonephritis,C12.777.419.570.363,Glomerulonephritis
3,Acute glomerulonephritis,C13.351.968.419.570.363,Glomerulonephritis
4,Acute hepatitis A,C02.440.420,Hepatitis A
...,...,...,...
601,Yellow fever,C02.081.980,Yellow Fever
602,Yellow fever,C02.782.350.250.980,Yellow Fever
603,Yellow fever,C02.782.417.881,Yellow Fever
605,Zika virus,C02.081.990,Zika Virus Infection


Matching GBD ID from Crosswalk

In [ ]:
GBD_match = []
GBD_ID = []

for cause in list(df_C['GBD disease cause']):
  if cause in GBD_causes:
    GBD_match.append('True')
    GBD_ID.append(int(df_GBD_sel[df_GBD_sel['Cause Name']==cause]['Cause ID']))
  else:
    GBD_match.append('False')
    GBD_ID.append('False')

df_C_GBD = df_C
df_C_GBD['GBD match'] = GBD_match
df_C_GBD['GBD ID'] = GBD_ID

df_C_GBD_match = df_C_GBD[df_C_GBD['GBD match'].str.contains('True')]
df_C_GBD_unmatch = df_C_GBD[df_C_GBD['GBD match'].str.contains('False')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Crosswalk & GBD Matched

In [ ]:
df_C_GBD_matched = df_C_GBD_match.drop('GBD match', axis=1)
df_C_GBD_matched

,GBD disease cause,MeSH code,MeSH term,GBD ID
0,Acne vulgaris,C17.800.030.150,Acne Vulgaris,661
1,Acne vulgaris,C17.800.794.111,Acne Vulgaris,661
2,Acute glomerulonephritis,C12.777.419.570.363,Glomerulonephritis,588
3,Acute glomerulonephritis,C13.351.968.419.570.363,Glomerulonephritis,588
4,Acute hepatitis A,C02.440.420,Hepatitis A,401
...,...,...,...,...
601,Yellow fever,C02.081.980,Yellow Fever,358
602,Yellow fever,C02.782.350.250.980,Yellow Fever,358
603,Yellow fever,C02.782.417.881,Yellow Fever,358
605,Zika virus,C02.081.990,Zika Virus Infection,935


In [ ]:
len(df_C_GBD_matched['GBD ID'].unique())

213

Saving Crosswalk & GBD Matched

In [ ]:
df_C_GBD_matched.to_csv('GBD Data & Crosswalk_matched.csv', index=False)

Crosswalk & GBD Un-matched

In [ ]:
df_C_GBD_unmatched = df_C_GBD_unmatch.drop('GBD match', axis=1)
df_C_GBD_unmatched

,GBD disease cause,MeSH code,MeSH term,GBD ID
66,Blindness and vision impairment,C10.597.751.941,Vision Disorders,False
67,Blindness and vision impairment,C11.966,Vision Disorders,False
68,Blindness and vision impairment,C23.888.592.763.941,VisionDisorders,False
69,Brain and nervous system cancer,C04.588.614.250.195,BrainNeoplasms,False
70,Brain and nervous system cancer,C04.588.614,Nervous SystemNeoplasms,False
71,Brain and nervous system cancer,C10.228.140.211,Brain Neoplasms,False
72,Brain and nervous system cancer,C10.551.240.250,Brain Neoplasms,False
73,Brain and nervous system cancer,C10.551,Nervous SystemNeoplasms,False
185,Epilepsy,C10.228.140.490,Epilepsy,False
242,H influenzae type B meningitis,C01.252.200.500.450,"Meningitis,Haemophilus",False


Saving Crosswalk & GBD Un-matched

In [ ]:
df_C_GBD_unmatched.to_csv('GBD Data & Crosswalk_unmatched.csv', index=False)

# **Integrated Map Ver.2 (GBD hierarchies & mesh_info.csv)**

Matching GBD ID with mesh_info.csv

In [ ]:
GBD_match = []
GBD_ID = []

for cause in list(mesh_C['name']):
  if cause in GBD_causes:
    GBD_match.append('True')
    GBD_ID.append(int(df_GBD_sel[df_GBD_sel['Cause Name']==cause]['Cause ID']))
  else:
    GBD_match.append('False')
    GBD_ID.append('False')

mesh_C_GBD = mesh_C
mesh_C_GBD['GBD match'] = GBD_match
mesh_C_GBD['GBD ID'] = GBD_ID

mesh_C_GBD_match = mesh_C_GBD[mesh_C_GBD['GBD match'].str.contains('True')]
mesh_C_GBD_match = mesh_C_GBD_match.drop('GBD match', axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
## Unique GBD IDs
print('Matched meSH & GBD ID:', len(mesh_C_GBD_match['GBD ID'].unique()))

Matched meSH & GBD ID: 44


Saving the Matched mesh_info.csv & GBD Data

In [ ]:
mesh_C_GBD_match.to_csv('GBD Data & mesh_info.csv_matched.csv', index=False)

Saving the Un-matched mesh_info.csv & GBD Data

In [ ]:
mesh_C_GBD_unmatch = mesh_C_GBD[mesh_C_GBD['GBD match'].str.contains('False')]
mesh_C_GBD_unmatch = mesh_C_GBD_unmatch.drop('GBD ID', axis=1)

In [ ]:
## Unique un-matched
print('Unique Un-matched:', len(mesh_C_GBD_unmatch['unique_identifier'].unique()))

Unique Un-matched: 4619


In [ ]:
mesh_C_GBD_unmatch.to_csv('GBD Data & mesh_info.csv_unmatched.csv', index=False)